# LLM Analysis Test

## 1. Imports and Setup

In [1]:
import sys
import os
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import LLM_MODEL_PATH
LLM_MODEL_PATH = os.path.abspath(os.path.join('..', LLM_MODEL_PATH))

c:\Users\ahmed\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Load the LLM

In [3]:
has_gpu = torch.cuda.is_available()
if has_gpu:
    device = torch.device("cuda")
    print("GPU is available. Using CUDA.")
else:
    device = torch.device("cpu")
    print("GPU not available. Using CPU.")

llm_model = None
llm_tokenizer = None
if has_gpu:
    try:
        print("Loading model in 4-bit for GPU.")
        quantization_config = BitsAndBytesConfig(load_in_4bit=True)
        llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_PATH)
        llm_model = AutoModelForCausalLM.from_pretrained(
            LLM_MODEL_PATH,
            quantization_config=quantization_config,
            torch_dtype=torch.float16,
            device_map="auto",
        )
        print("Analysis LLM loaded successfully in 4-bit.")
    except Exception as e:
        print(f"Error loading Analysis LLM on GPU: {e}")
else:
    print("Skipping LLM loading as no GPU is available.")

GPU is available. Using CUDA.
Loading model in 4-bit for GPU.


Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.63s/it]

Analysis LLM loaded successfully in 4-bit.


## 3. Prepare the Article and Prompt

In [4]:
with open('../article.txt', 'r') as f:
    article_text = f.read()

## 4. Run Inference

In [5]:
analysis_start = "<start_analysis>"
analysis_end = "</end_analysis>"
system_prompt = f"""You are an analyst given an article and associated entities.
Analyze the article and produce:
- Concise Summary: A brief overview.
- Expanded Analysis: A deeper analysis.
- Impact Forecast: List impacts with type, horizon (SHORT_TERM/MEDIUM_TERM/LONG_TERM), magnitude (LOW/MEDIUM/HIGH), probability (0-1), and rationale.
- Scenarios: List possible outcomes with probability (0-1) and description.
- Confidence: Overall confidence score (0-1).
- Evidence Refs: List of reference IDs.

Place your full analysis between {analysis_start} and {analysis_end}.
Output in plain text with sections like 'Concise Summary:', 'Impact Forecast:', etc."""

user_prompt = f"""Article Text: {article_text}\n\nAnalyze this article."""

prompt = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

if not llm_tokenizer or not llm_model:
    if not has_gpu:
        print("LLM analysis skipped: No GPU available.")
    else:
        print("LLM model or tokenizer not available.")
else:
    prompt_text = llm_tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    inputs = llm_tokenizer(prompt_text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=2048,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=llm_tokenizer.eos_token_id,
            eos_token_id=llm_tokenizer.eos_token_id
        )

    generated_text = llm_tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    match = re.search(rf"{re.escape(analysis_start)}\s*(.+?)\s*{re.escape(analysis_end)}", generated_text, re.DOTALL | re.IGNORECASE)
    if match:
        analysis = match.group(1).strip()
        print(analysis)
    else:
        print(f"Raw Generated Output (no tags found):\n{generated_text}")

Raw Generated Output (no tags found):
<start_analysis>

**Concise Summary:**
Salesforce has agreed to acquire Slack Technologies for approximately $27.7 billion in cash and stock, marking its largest-ever acquisition. The deal combines Salesforce's Customer 360 platform with Slack's channel-based messaging platform, aiming to create a comprehensive 'operating system for your workday' and challenge Microsoft's Teams product.

**Expanded Analysis:**
The acquisition is a strategic move by Salesforce to strengthen its position in the growing market for team communication tools. Slack's integration with Salesforce's existing platform will enhance customer experience, increase stickiness, and make it harder for competitors to poach clients. The deal also underscores the trend of software companies acquiring complementary technologies to expand their offerings and increase market share. The acquisition's significance is further amplified by the fact that Slack is a beloved communication tool 